# Filtering Data with `.query()`

The `Flow` class includes a powerful `.query()` method, enabling you to filter records using a concise, string-based expression. This improves the readability and flexibility of your data processing pipelines.

You can specify complex filtering conditions using standard Python comparison and logical operators, field access, and even built-in functions or string methods.

Use `.query()` when you want to:

- Prototype filters quickly.
- Write more readable and maintainable pipeline logic.
- Let users define filters without writing custom Python code.

---

## How it Works

The `.query()` method parses a string expression using Python's Abstract Syntax Tree (AST) module. It then converts this AST into an efficient predicate function, which is applied to each record in the `Flow` stream. This approach ensures security (as it doesn't use `eval()` directly on arbitrary input) and allows for robust validation of the query syntax.

---

## Key Capabilities & Supported Syntax

The `.query()` method supports a rich set of expressions:

### 1. Basic Comparisons

You can compare record fields to literal values using standard comparison operators:

- `==` (equals)
- `!=` (not equals)
- `>` (greater than)
- `>=` (greater than or equal to)
- `<` (less than)
- `<=` (less than or equal to)

```python
# Example: Filter for matches with more than 3 goals
flow.query("goals > 3")

# Example: Filter for matches played by 'Man City'
flow.query("home_team == 'Man City'")
```

### 2. Logical Operators

Combine multiple conditions using `and`, `or`, and `not`:

- `and`
- `or`
- `not`

```python
# Example: Home wins for 'Liverpool'
flow.query("home_team == 'Liverpool' and home_goals > away_goals")

# Example: Matches not involving 'Arsenal'
flow.query("not (home_team == 'Arsenal' or away_team == 'Arsenal')")
```

### 3. Field Access

Access nested fields using dot notation:

- `field.subfield`

```python
# Example: Filter based on nested 'venue.city' field
flow.query("venue.city == 'London'")
```

### 4. Membership Operators (`in`, `not in`)

Check if a field's value is present in a list or tuple:

- `in`
- `not in`

```python
# Example: Filter for matches involving specific teams
flow.query("home_team in ['Chelsea', 'Tottenham']")

# Example: Filter for matches NOT involving a specific league
flow.query("league not in ['Premier League', 'La Liga']")
```

> ⚠️ `in` and `not in` require the field to appear on the left-hand side of the expression. Reverse usage is not currently supported (e.g., "Man City" in home_team will raise an error).

### 5. `is None` and `is not None`

Check for null/missing values:

* `is None`
* `is not None`

```python
# Example: Find records where 'player.injury_status' is null
flow.query("player.injury_status is None")

# Example: Find records where 'player.injury_status' is not null
flow.query("player.injury_status is not None")
```

### 6. String Methods

Apply common string transformations for comparison. Note these are used *within* a comparison:

- `len()`: Get the length of a string or list/tuple.
- `.lower()`: Convert a string to lowercase.
- `.upper()`: Convert a string to uppercase.

```python
# Example: Find teams whose name is exactly 'manchester united' (case-insensitive)
flow.query("home_team.lower() == 'manchester united'")

# Example: Find teams with a short name
flow.query("len(home_team) < 8")
```

### 7. Predicate-Style String Methods (Standalone)

Directly check string properties using method calls:

- `.contains(substring)`
- `.startswith(prefix)`
- `.endswith(suffix)`
- `.regex(pattern, flags=0)` or `.match(pattern, flags=0)`

```python
import re

# Example: Find home teams containing 'united'
flow.query("home_team.contains('united')")

# Example: Find away teams starting with 'West'
flow.query("away_team.startswith('West')")

# Example: Find matches with a score like '2-X' using regex
# Note: Regex flags like re.IGNORECASE can be passed using @var references
flow.query("score.regex(@pattern, @flags)", local_vars={'pattern': r"2-\d", 'flags': re.IGNORECASE})
```

### 8. Referencing Local Python Variables (`@var`)

To make your queries dynamic, you can inject external Python variables using the `@` symbol. This allows you to construct queries programmatically while maintaining readability. For example, `@team_name` will be replaced with the actual value of the variable `team_name` from your Python scope.

This is especially useful when working with date ranges, parameterized filters, or reusable queries.

```python
import datetime

min_goals = 2
team_name = "Liverpool"
start_date = datetime.date(2023, 1, 1)

# Example: Using numeric and string variables
flow.query("home_goals >= @min_goals and home_team == @team_name")

# Example: Using a date object
flow.query("match_date >= @start_date")
```

For regular expressions, you can pass flags such as `re.IGNORECASE` or `re.MULTILINE` by referencing them the same way:

```python
import re
flow.query("home_team.regex(@pattern, @flags)", local_vars={'pattern': r"arsenal", 'flags': re.IGNORECASE})
```

Remember:

- Always provide external variables via `local_vars`.
- Regex flags must be passed as values from the `re` module.
- The query parser substitutes `@var` with safe, scoped values - no arbitrary code execution occurs.

### 9. Function Calls (Limited)

Currently, only `datetime()` and `date()` from the `datetime` module are supported for direct use within the query string to create date objects for comparison. This will be expanded in the future.

```python
# Example: Matches after a specific date
flow.query("match_date > date(2024, 6, 30)")
```

---

## Usage Examples

Let's assume you have a Flow object (e.g., loaded from MatchFlow) containing records with fields like `home_team`, `away_team`, `home_goals`, `away_goals`, `match_date`, `venue.city`, etc.

```python
from penaltyblog import MatchFlow
import datetime

# Example 1: Simple filter for specific teams and results
# Replace 'your_match_data.json' with your actual data source if loading from file
flow = MatchFlow(data_source=[{'home_team': 'Manchester United', 'away_team': 'Liverpool', 'home_goals': 2, 'away_goals': 1, 'match_date': datetime.date(2024, 5, 1), 'venue': {'city': 'Manchester'}, 'score': '2-1'},
                            {'home_team': 'Liverpool', 'away_team': 'Arsenal', 'home_goals': 0, 'away_goals': 1, 'match_date': datetime.date(2024, 4, 15), 'venue': {'city': 'Liverpool'}, 'score': '0-1'},
                            {'home_team': 'Chelsea', 'away_team': 'Tottenham', 'home_goals': 3, 'away_goals': 3, 'match_date': datetime.date(2024, 3, 10), 'venue': {'city': 'London'}, 'score': '3-3'},
                            {'home_team': 'Arsenal', 'away_team': 'Man City', 'home_goals': 1, 'away_goals': 0, 'match_date': datetime.date(2024, 2, 20), 'venue': {'city': 'London'}, 'score': '1-0'}
                            ]) # Using sample data for demonstration

man_utd_wins_at_home = (
    flow.query("home_team == 'Manchester United' and home_goals > away_goals")
    .collect()
)
print(f"Man Utd home wins: {len(man_utd_wins_at_home)}")

# Example 2: Using a variable for a dynamic date filter
today = datetime.date.today()
recent_matches = (
    flow.query("match_date >= @today", local_vars={'today': today})
    .collect()
)
print(f"Recent matches (since {today}): {len(recent_matches)}")

# Example 3: Combining string methods and logical operators
london_derbies = (
    flow.query("(home_team.contains('London') or away_team.contains('London')) and venue.city == 'London'")
    .collect()
)
print(f"London derbies: {len(london_derbies)}")

# Example 4: Chained comparisons and null checks
# Add a record with null referee for this example
flow_with_null = MatchFlow(data_source=flow.collect() + [{'home_team': 'Everton', 'away_team': 'Fulham', 'home_goals': 2, 'away_goals': 2, 'match_date': datetime.date(2024, 1, 5), 'match_details': {'referee': None}, 'score': '2-2'}])

high_scoring_draws = (
    flow_with_null.query("home_goals > 1 and home_goals == away_goals and match_details.referee is not None")
    .collect()
)
print(f"High scoring draws with known referee: {len(high_scoring_draws)}")

# Example 5: Using regex for advanced pattern matching
import re
pattern_for_specific_teams = r"(Chelsea|Arsenal)"
ignore_case_flag = re.IGNORECASE

filtered_by_regex = (
    flow.query("home_team.regex(@pattern, @flags) or away_team.regex(@pattern, @flags)",
               local_vars={'pattern': pattern_for_specific_teams, 'flags': ignore_case_flag})
    .collect()
)
print(f"Matches involving Chelsea or Arsenal (case-insensitive regex): {len(filtered_by_regex)}")
```